# Bigram Language Model

In this lab we will implement a bigram language model and use it to compute the probability of some sample sentences.

As you go through, make sure you understand what's going on in each cell, and ask if it is unclear.

### Outcomes

- Know how to count word frequencies in a corpus using Python libraries.
- Understand how to compute conditional probabilities.
- Be able to apply the chain rule to compute the probability of a sentence.

### Overview

The first part of the notebook loads the same dataset as last week.
The next part splits the data into training and test sets, and tokenises the utterances.
After this there are some tasks to complete to implement and test the language model.


# 1. Preparing the Data


In [2]:
import json


def generate_examples(filepath: str):
    """
    Adapted from https://huggingface.co/datasets/doc2dial/blob/main/doc2dial.py#L226
    """
    with open(filepath, encoding="utf-8") as f:
        data = json.load(f)
        for domain in data["dial_data"]:
            for doc_id in data["dial_data"][domain]:
                for dialogue in data["dial_data"][domain][doc_id]:
                    x = {
                        "dial_id": dialogue["dial_id"],
                        "domain": domain,
                        "doc_id": doc_id,
                        "turns": dialogue["turns"],
                    }

                    yield dialogue["dial_id"], x


dataset = [
    *generate_examples(
        # The filepath of the training data from the repository:
        # https://github.com/doc2dial/sharedtask-dialdoc2021
        "../../../sharedtask-dialdoc2021/data/doc2dial/v1.0.1/doc2dial_dial_train.json",
    )
]

In [3]:
# Collect all of the utterances into a list. For this task, we don't care about
# the order of the utterances in the conversation - we will just use them as
# examples of the language we want to model.
docs = [
    turn["utterance"] for item in dataset[:100] for turn in item[1]["turns"]
]

print(f"Number of utterances: {len(docs)}")

Number of utterances: 1195


In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.4 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
from pprint import pprint
from nltk.lm.preprocessing import pad_both_ends
import spacy
import typing

nlp = spacy.load("en_core_web_sm")

# Tokenize the utterances.
docs_tokenized_unpadded = [[token.text for token in nlp(doc)] for doc in docs]

token_start = "<s>"
token_end = "</s>"


# Add artificial start `<s>` and end `</s>` tokens to each utterance. These will
# be used to compute the conditional probabilities of each word at the start of
# a sentence and the conditional probabilities of ending the sentence after a
# particular word. This lets us model which words are most likely to start or
# end a sentence.
def pad(docs: typing.List[typing.List[str]]) -> typing.List[typing.List[str]]:
    return [[token_start] + doc + [token_end] for doc in docs]


docs_tokenized = pad(docs_tokenized_unpadded)

# Print an example of a tokenized utterance.
doc_tokenized_example_1 = docs_tokenized[2]
print("Our result:")
pprint(doc_tokenized_example_1)

Our result:
['<s>', 'Can', 'I', 'do', 'my', 'DMV', 'transactions', 'online', '?', '</s>']


In [6]:
# Compare the result to NLTK's.
docs_tokenized_nltk = [
    pad_both_ends(
        doc_tokenized_unpadded,
        n=2,
    )
    for doc_tokenized_unpadded in docs_tokenized_unpadded
]
print("NLTK's result:")
pprint(list(docs_tokenized_nltk[2]))

NLTK's result:
['<s>', 'Can', 'I', 'do', 'my', 'DMV', 'transactions', 'online', '?', '</s>']


In [7]:
from pprint import pprint

# from sklearn.model_selection import train_test_split

# Split the data into training and test sets with `scikit-learn`. Note that we
# split the unpadded documents *and then* pad them so that we use the same split
# to evaluate our results and NLTK's.
# data_train_unpadded, data_test_unpadded = train_test_split(
#     docs_tokenized_unpadded, train_size=0.8, test_size=0.2
# )
data_train_unpadded = docs_tokenized_unpadded
data_test_unpadded = []
data_train = pad(data_train_unpadded)
data_test = pad(data_test_unpadded)

pprint(
    f"The training set has {len(data_train)} samples and the test set has {len(data_test)} samples."
)

'The training set has 1195 samples and the test set has 0 samples.'


# 2. Counting Tokens

The bigram language model needs to compute two sets of counts from the training data:

1. The counts of how many times each bigram occurs.
2. The counts of how many times each word type occurs as the first token in a bigram.

Let's start by finding the vocabulary of unique token 'types':


In [8]:
import numpy as np
from pprint import pprint

vocab = np.unique(np.concatenate(data_train))
vocab_size = len(vocab)

pprint(vocab)
pprint(f"There are {vocab_size} types in our vocabulary.")

# So far, `vocab` is a numpy array. It may be simpler to work with a list:
vocab = list(vocab.tolist())

array(['\n', '\n   ', ' ', ..., 'yourself', '¡', '´'], dtype='<U40')
'There are 1838 types in our vocabulary.'


In [9]:
from pprint import pprint
import typing


# For example, we can find the index of a token like so:
def find_token_index(token: str, vocab: list[str]):
    try:
        return vocab.index(token)
        # If `vocab` were a numpy array, we could find the token's index like so:
        # `return np.argwhere(vocab == token)[0][0]`
    except ValueError:
        return -1


pprint(find_token_index(token_start, vocab))

92


In [10]:
import numpy.typing as npt


# Define a function to print the bigram statistics of a tokenized utterance.
def print_bigram_statistics(
    bigram_matrix: npt.NDArray, tokenized: typing.List[str]
):
    bigram_statistics = []
    for token_current, token_next in zip(
        tokenized,
        tokenized[1:],
    ):
        # Find the indices of the tokens in the vocabulary.
        token_current_index = find_token_index(token_current, vocab)
        token_next_index = find_token_index(token_next, vocab)

        # If both tokens are in the vocabulary, find the value of the statistic
        # for the bigram and add it to the list to print.
        if token_current_index != -1 and token_next_index != -1:
            bigram_statistic = bigram_matrix[
                token_current_index, token_next_index
            ]
            bigram_statistics.append(
                f"{token_current} {token_next}: {bigram_statistic}"
            )

    pprint(bigram_statistics)

**TODO 2.1:** count the bigrams that occur in the training set.


In [11]:
def find_bigram_counts(
    utterances: typing.List[typing.List[str]], vocab_size: int
):
    # A matrix whose row indices correspond to the first tokens in bigrams and
    # column indices correspond to the second tokens in bigrams. The indices must
    # map to the index of the token in the vocabulary. The values of the matrix will
    # be the token counts. We initialize the matrix with ones to use add-one
    # smoothing.
    bigram_counts = np.ones((vocab_size, vocab_size))

    tokens = [token for utterance in utterances for token in utterance]

    # Iterate the tokens in each utterance pairwise.
    for token_current, token_next in zip(tokens, tokens[1:]):
        # Find the indices of the tokens in the vocabulary.
        token_current_index = find_token_index(token_current, vocab)
        token_next_index = find_token_index(token_next, vocab)

        # If both tokens are in the vocabulary, increment the bigram count.
        if token_current_index != -1 and token_next_index != -1:
            bigram_counts[token_current_index, token_next_index] += 1

    return bigram_counts


bigram_counts = find_bigram_counts(data_train, vocab_size)

# Print the counts of the bigrams in an example of a tokenized utterance.
print("Our result:")
print_bigram_statistics(bigram_counts, doc_tokenized_example_1)

Our result:
['<s> Can: 48.0',
 'Can I: 28.0',
 'I do: 52.0',
 'do my: 4.0',
 'my DMV: 4.0',
 'DMV transactions: 3.0',
 'transactions online: 2.0',
 'online ?: 9.0',
 '? </s>: 484.0']


In [12]:
from nltk.lm.models import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import bigrams
from pprint import pprint

# Compare the result to NLTK's.
laplace = Laplace(order=2)
laplace.fit(*padded_everygram_pipeline(order=2, text=data_train_unpadded))

print("NLTK's result:")
pprint(
    [
        # NLTK doesn't include the pseudo-counts introduced by Laplace (add-one)
        # or Lidstone (add-k) smoothing in its `counts` attribute, so add one.
        # It does include the pseudo-counts in its `unmasked_score` method (and,
        # therefore, the dependent `score`, `logscore`, etc. methods.
        # See here: https://www.nltk.org/_modules/nltk/lm/models.html#Lidstone.unmasked_score
        f"{token_current} {token_next}: {laplace.counts[[token_current]][token_next] + 1}"
        for token_current, token_next in list(bigrams(doc_tokenized_example_1))
    ]
)

NLTK's result:
['<s> Can: 48',
 'Can I: 28',
 'I do: 52',
 'do my: 4',
 'my DMV: 4',
 'DMV transactions: 3',
 'transactions online: 2',
 'online ?: 9',
 '? </s>: 484']


**TODO 2.2:** Apply numpy's sum() function to the 'counts' variable to compute the number of times each word type occurs as the first token in a bigram.


In [13]:
from pprint import pprint

# `axis=1`: apply the operation row-wise, i.e., across all rows for each column.
first_token_counts = bigram_counts.sum(axis=1)


# Compute the unigram counts. The result should be the same.
def find_unigram_counts(
    utterances: typing.List[typing.List[str]], vocab_size: int
):
    # A vector whose indices correspond to the tokens in the vocabulary.
    unigram_counts = np.zeros(vocab_size)

    # Iterate the tokens in the utterances.
    for token in [token for utterance in utterances for token in utterance]:
        # Find the index of the token in the vocabulary.
        token_index = find_token_index(token, vocab)

        # If the token is in the vocabulary, increment the unigram count.
        if token_index != -1:
            unigram_counts[token_index] += 1

    return unigram_counts


unigram_counts = find_unigram_counts(data_train, vocab_size)

# Print the counts with which the tokens in an example of a tokenized utterance
# occur as the first token in a bigram.
print("\nBigram first-token counts:")
pprint(
    [
        f"{token}: {first_token_counts[find_token_index(token, vocab)]}"
        for token in doc_tokenized_example_1
    ]
)

# Print the unigram counts, which are smaller than the bigram first-token counts
# by the size of the vocabulary.
print("\nUnigram counts:")
pprint(
    [
        f"{token}: {unigram_counts[find_token_index(token, vocab)]}"
        for token in doc_tokenized_example_1
    ]
)


Bigram first-token counts:
['<s>: 3033.0',
 'Can: 1894.0',
 'I: 2367.0',
 'do: 2034.0',
 'my: 2112.0',
 'DMV: 2048.0',
 'transactions: 1848.0',
 'online: 1884.0',
 '?: 2353.0',
 '</s>: 3032.0']

Unigram counts:
['<s>: 1195.0',
 'Can: 56.0',
 'I: 529.0',
 'do: 196.0',
 'my: 274.0',
 'DMV: 210.0',
 'transactions: 10.0',
 'online: 46.0',
 '?: 515.0',
 '</s>: 1195.0']


In [14]:
print("NLTK's result:")
print("\nUnigram counts:")
pprint(
    [
        f"{token}: {laplace.counts.unigrams[token]}"
        for token in doc_tokenized_example_1
    ]
)

# Check our results are the same.
for index, count in enumerate(unigram_counts):
    if count != laplace.counts.unigrams[vocab[index]]:
        print(
            f"{vocab[index]}: {count} != {laplace.counts.unigrams[vocab[index]]}"
        )

NLTK's result:

Unigram counts:
['<s>: 1195',
 'Can: 56',
 'I: 529',
 'do: 196',
 'my: 274',
 'DMV: 210',
 'transactions: 10',
 'online: 46',
 '?: 515',
 '</s>: 1195']


**TODO 2.3:** Compute a matrix (numpy array) of conditional probabilities using the counts. Compute the log of this matrix as a variable 'log_cond_probs'.


In [15]:
import numpy.typing as npt
import typing
from pprint import pprint


# Define a function to print the bigram statistics of a tokenized utterance.
def print_bigram_statistics(
    bigram_matrix: npt.NDArray, tokenized: typing.List[str]
):
    bigram_statistics = []
    for token_current, token_next in zip(
        tokenized,
        tokenized[1:],
    ):
        # Find the indices of the tokens in the vocabulary.
        token_current_index = find_token_index(token_current, vocab)
        token_next_index = find_token_index(token_next, vocab)

        # If both tokens are in the vocabulary, find the value of the statistic
        # for the bigram and add it to the list to print.
        if token_current_index != -1 and token_next_index != -1:
            bigram_statistic = bigram_matrix[
                token_current_index, token_next_index
            ]
            bigram_statistics.append(
                f"{token_current} {token_next}: {bigram_statistic}"
            )

    pprint(bigram_statistics)


# Compute the bigram conditional probabilities.
bigram_conditional_probabilities = np.divide(bigram_counts, first_token_counts)

print("Our results:")
print("\nBigram conditional probabilities:")
print_bigram_statistics(
    bigram_conditional_probabilities, doc_tokenized_example_1
)

# Compute the logarithms of the bigram conditional probabilities.
bigram_conditional_log_probabilities = np.log(bigram_conditional_probabilities)

print("\nLogarithms of bigram conditional probabilities:")
print_bigram_statistics(
    bigram_conditional_log_probabilities, doc_tokenized_example_1
)

Our results:

Bigram conditional probabilities:
['<s> Can: 0.025343189017951427',
 'Can I: 0.011829319814110688',
 'I do: 0.025565388397246803',
 'do my: 0.001893939393939394',
 'my DMV: 0.001953125',
 'DMV transactions: 0.0016233766233766235',
 'transactions online: 0.0010615711252653928',
 'online ?: 0.003824904377390565',
 '? </s>: 0.15963060686015831']

Logarithms of bigram conditional probabilities:
['<s> Can: -3.6752452628381325',
 'Can I: -4.4371740993387805',
 'I do: -3.666515858027078',
 'do my: -6.269096283706261',
 'my DMV: -6.238324625039508',
 'DMV transactions: -6.423246963533519',
 'transactions online: -6.848005274576363',
 'online ?: -5.566221811391143',
 '? </s>: -1.8348928400456306']


In [21]:
from nltk.util import bigrams
from pprint import pprint

# Compare the result to NLTK's.
# TODO: Why are results different?
print("NLTK's result:")
print("\nBigram probabilities:")
pprint(
    [
        f"{token_current} {token_next}: {laplace.score(token_next, [token_current])}"
        for token_current, token_next in list(bigrams(doc_tokenized_example_1))
    ]
)

print("\nBigram log probabilities:")
pprint(
    [
        f"{token_current} {token_next}: {laplace.logscore(token_next, [token_current])}"
        for token_current, token_next in list(bigrams(doc_tokenized_example_1))
    ]
)

NLTK's result:

Bigram probabilities:
['<s> Can: 0.015820698747528016',
 'Can I: 0.014775725593667546',
 'I do: 0.02195945945945946',
 'do my: 0.0019656019656019656',
 'my DMV: 0.001893043066729768',
 'DMV transactions: 0.0014641288433382138',
 'transactions online: 0.001081665765278529',
 'online ?: 0.004774535809018567',
 '? </s>: 0.205607476635514']

Bigram log probabilities:
['<s> Can: -5.982042869525877',
 'Can I: -6.0806272110008495',
 'I do: -5.5090136474878575',
 'do my: -8.990813079153611',
 'my DMV: -9.04507705193494',
 'DMV transactions: -9.415741768290092',
 'transactions online: -9.852529509404196',
 'online ?: -7.710423806713715',
 '? </s>: -2.2820353677638496']


**TODO 2.4:** Write a function that uses log_cond_probs to compute the probability of a given tokenised sentence, such as the example below.


In [17]:
# An example of a tokenized utterance.
doc_tokenized_example_2 = [
    "<s>",
    "If",
    "you",
    "give",
    "me",
    "the",
    "help",
    ",",
    "what",
    "is",
    "the",
    "payment",
    "system",
    "?",
    "<e>",
]

In [18]:
import numpy as np
import numpy.typing as npt
from pprint import pprint


def find_log_probability(
    tokenized: typing.List[str],
    vocab: typing.List[str],
    bigram_matrix: npt.ArrayLike,
):
    log_probability = 0.0
    for token_current, token_next in zip(
        tokenized,
        tokenized[1:],
    ):
        # Find the indices of the tokens in the vocabulary.
        token_current_index = find_token_index(token_current, vocab)
        token_next_index = find_token_index(token_next, vocab)

        # If both tokens are in the vocabulary, add to the log probability.
        if token_current_index != -1 and token_next_index != -1:
            log_probability += bigram_matrix[
                token_current_index, token_next_index
            ]

    return log_probability


pprint(
    np.exp(
        find_log_probability(
            doc_tokenized_example_2, vocab, bigram_conditional_log_probabilities
        )
    )
)

2.3643417950084365e-35


**TODO 2.5:** Compute the perplexity over the whole test set. You will need to make sure your code can handle unknown words -- make sure it does not end up misusing the index of -1 returned by get_index_for_word() for unknown words.


In [32]:
import math


def find_perplexity(
    tokenized: typing.List[str],
    vocab: typing.List[str],
    bigram_matrix: npt.ArrayLike,
):
    """
    The perplexity is the Nth root of the product of the inverse probabilities
    of the bigrams, where N is the number of bigrams. Because of the properties
    of logarithms, this is equivalent to (pseudocode):
    ```
        exp(1 - sum(log(probability(bigram)))/N)
    ```
    """
    return math.exp(
        1
        - find_log_probability(tokenized, vocab, bigram_matrix)
        / (len(tokenized) - 1)
    )


# Flatten the test set into a single "document".
data_test = pad(
    [[token.text for token in nlp("Can I do my DMV transactions online?")]]
)
print(data_test)

print("Our result:")
pprint(
    find_perplexity(data_test[0], vocab, bigram_conditional_log_probabilities)
)

[['<s>', 'Can', 'I', 'do', 'my', 'DMV', 'transactions', 'online', '?', '</s>']]
Our result:
401.5827718678161


In [31]:
print("NLTK's result:")
pprint(laplace.perplexity(data_test[0]))

NLTK's result:
909.1327830154821


**EXTENSION 1:** Use the language model to generate new sentences by sampling.
You can follow the example below to sample using scipy's multinomial class. Replace the distribution with the conditional distribution we computed earlier.


In [ ]:
from scipy.stats import multinomial

example_vocab = np.array(["a", "b", "c", "d"])

distribution = [0.3, 0.2, 0.1, 0.4]
sample = multinomial.rvs(1, distribution)
sample_bool = sample.astype(bool)  # convert the sample from integer to boolean
generated_word = example_vocab[sample_bool][
    0
]  # use the one-hot boolean vector to look up the word

print(generated_word)

In [ ]:
current_tok = "<s>"
tokens = [current_tok]

while current_tok != "<e>" and len(tokens) < 1000:
    ### WRITE YOUR CODE HERE

    ###
    tokens.append(current_tok)

print(tokens)
print(len(tokens))

MORE EXTENSIONS:

- Add some smoothing to the counts and see how it affects the results.
- Use trigrams instead of bigrams. Does it improve perplexity?
